# Preprocessing del dataset

## Import delle librerie

In [ ]:
from handcrafted.app.features.plotter.frames_plotter import plot_frames
from handcrafted.app.dataset.dataset import Dataset
from handcrafted.app.preprocess.roi_extractor import RoiExtractor
from handcrafted.app.utilities.utils import create_mp4_video_from_frames

import cv2

## Caricamento del dataset originale
Attraverso la classe [Dataset](./handcrafted/app/dataset/dataset.py), vengono caricati, dal file [WLASL_v0.3.json](./data/WLASL_v0.3.json), tutti i video presenti nel dataset. Con il parametro `only_samples` è possibile specificare se caricare tutti i video presenti nella cartella `data/videos` (quelli già preprocessati) o solo un campione (composto da due soli video) presi dal dataset originale, e situati nella cartella `data/original_videos_sample`.

In [ ]:
dataset = Dataset("data/WLASL_v0.3.json", only_samples=True)
video1 = dataset.videos[0]
video2 = dataset.videos[1]

## Modifica del parametro `frame_end`
Dato che la maggior parte dei valori di `frame_end` è pari a `-1`, si è preferito modificare questo valore per rispecchiare il numero di frame di cui è composto il video. Per aggiornare il parametro `frame_end` è stato realizzato il metodo `update_videos` nella classe [Dataset](./handcrafted/app/dataset/dataset.py), il cui unico compito è aggiornare il file json `data/WLASL_v0.3.json` con i nuovi valori.

In [ ]:
dataset.update_videos()

## Visualizzazione dei video originali
Il metodo `plot_frames` permette di visualizzare i frame di un video.

In [ ]:
plot_frames(video1.get_frames())
plot_frames(video2.get_frames())

## Estrazione della ROI
Verrà estratta la Region of Interest per entrambi i video. Il video viene poi ridimensionato a 224x224 aggiungendo padding se necessario.
Per svolgere questo compito è stata realizzata la classe [RoiExtractor](handcrafted/app/preprocess/roi_extractor.py).

In [ ]:
video1_roi_extractor = RoiExtractor(
    frames=video1.get_frames(), bbox=video1.bbox, resize=224
)
video1_frames = video1_roi_extractor.extract(False)
plot_frames(video1_frames)

In [ ]:
video2_roi_extractor = RoiExtractor(
    frames=video2.get_frames(), bbox=video2.bbox, resize=224
)
video2_frames = video2_roi_extractor.extract(False)
plot_frames(video2_frames)

## Rimozione del background
Verrà rimossa la parte di background dai video, utilizzando remove di rembg.

In [ ]:
frames_1_no_bg = video1.get_frames(remove_bg=True)
plot_frames(frames_1_no_bg)
frames_1_no_bg = [cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) for frame in frames_1_no_bg]
create_mp4_video_from_frames(
    frames_1_no_bg,
    video1.fps,
    f"data/original_videos_sample/no_bg/{video1.video_id}.mp4",
)

## Salvataggio dei video processati
Vengono quindi salvati i video, utilizzando ffmpeg, nella cartella `data/original_videos_sample/updated`.

In [ ]:
frames_bgr = [
    cv2.cvtColor(frame, cv2.COLOR_RGB2BGR) for frame in video1_frames
]
create_mp4_video_from_frames(
    frames_bgr,
    video1.fps,
    f"data/original_videos_sample/updated/{video1.video_id}.mp4",
)

In [ ]:
frames_bgr = [
    cv2.cvtColor(frame, cv2.COLOR_RGB2BGR) for frame in video2_frames
]
create_mp4_video_from_frames(
    frames_bgr,
    video2.fps,
    f"data/original_videos_sample/updated/{video2.video_id}.mp4",
)